In [29]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import time
from splinter import Browser
import pandas as pd

In [3]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/Users/thomasadamson/Downloads/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

## NASA Mars News

In [4]:
#Start browser session for mars.nasa.gov/news/
browser = init_browser()

url = "https://mars.nasa.gov/news/"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [5]:
#Use class 'content_title to retrieve article title'
title = soup.find_all('div', class_= 'content_title')[1].text
title

"New Study Challenges Long-Held Theory of Fate of Mars' Water"

In [6]:
#Use class 'article_teaser_body to retrieve article tag'
tag = soup.find_all('div', class_='article_teaser_body')[0].text
tag

'The new science results indicate that a large quantity of the Red Planet’s water is trapped in its crust rather than having escaped into space.'

In [7]:
browser.quit()

## JPL Mars Space Images - Featured Image

In [8]:
#Start browser session for mars.nasa.gov/news/
browser = init_browser()

url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/"
browser.visit(url + "index.html")

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [22]:
#Use img tag and class to find pathing to image
image_path = soup.find('img', class_ = 'headerimage')['src']

In [25]:
#Full url includes both the initial page and the pathing
image_url = url + image_path

In [26]:
image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

In [27]:
browser.quit()

## Mars Facts

In [28]:
#Define url for pandas scrape
url= 'https://space-facts.com/mars/'

In [31]:
tables = pd.read_html(url)
tables

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [32]:
mars_facts = tables[0]

In [33]:
mars_facts_html = mars_facts.to_html()
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    <

## Mars Hemispheres

In [110]:
#Start browser session 
browser = init_browser()

url = "https://astrogeology.usgs.gov"
browser.visit(url + "/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [111]:
hemispheres = soup.find_all('div', class_ = 'item')

In [112]:
#append each hemisphere's link title into a list so we can click them in splinter
pages = []
for title in hemispheres:
    pages.append(title.find('h3').text)
    
pages

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [114]:
#Loop through each page and get image url

for image_page in pages:
    url = "https://astrogeology.usgs.gov"
    browser.visit(url + "/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")
    
    browser.links.find_by_partial_text(image_page).click()